<a href="https://colab.research.google.com/github/turkgpt/finetuning/blob/main/turk_gpt_finetuning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "OmBayus/Turk-GPT-v0.2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    # load_in_4bit = load_in_4bit,
    #token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
OmBayus/Turk-GPT-v0.2 does not have a padding token! Will use pad_token = <unk>.


LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep

In [ ]:
alpaca_prompt = """Below is a system message that describes a task, paired with a prompt that provides further context. Write a response that appropriately completes the request.

### System:
{}

### Prompt:
{}

### Chosen Response:
{}

### Rejected Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    systems = examples["system"]
    prompts = examples["prompt"]
    chosens = examples["chosen"]
    rejecteds = examples["rejected"]
    texts = []
    for system, prompt, chosen, rejected in zip(systems, prompts, chosens, rejecteds):
        # Create the text using the new template and add the EOS token
        text = alpaca_prompt.format(system, prompt, chosen, rejected) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}
pass

from datasets import load_dataset
# Load the new dataset
dataset = load_dataset("jondurbin/truthy-dpo-v0.1", split="train")

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
alpaca_prompt = """Below is a text in Turkish. Please complete the text appropriately.

### Text:
{}

### Completion:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    texts = examples["text"]
    formatted_texts = []

    for text in texts:
        # Metni ikiye bölerek ilk kısmını girdi, tamamını çıktı olarak kullanın
        split_index = len(text) // 2
        input_text = text[:split_index]
        output_text = text

        formatted_text = alpaca_prompt.format(input_text, output_text) + EOS_TOKEN
        formatted_texts.append(formatted_text)

    return { "text" : formatted_texts }
pass

from datasets import load_dataset
dataset = load_dataset("OmBayus/turkgpt_dataset",data_dir="data5",split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10001 [00:00<?, ? examples/s]

### Train the model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/200023 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 200,023 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 200
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,6.517200
2,7.108200
3,5.376000
4,4.437500
5,3.814900
6,3.081100
7,2.470400
8,2.286500
9,1.861600
10,1.865200


###  Test Model with Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> \nContinue the fibonnaci sequence.\n\nThe Fibonacci sequence is a series of numbers where a number is the sum of the two preceding ones, starting from 0 and 1. That is,\n\n0, 1, 1, 2, 3, 5, 8, 13, ']

### TextStreamer

In [ ]:
alpaca_prompt = """
{}"""
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Nasılsın", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<s><|im_start|> system
Your name is Turk-GPT. You made by OmBayus.<|im_end|> 
<|im_start|> user
Senin ismin nedir?<|im_end|> 
<|im_start|> assistant
İsmim Turk-GPT ve yaratıcım OmBayus tarafından oluşturuldu.<|im_end|>


AttributeError: 'TextStreamer' object has no attribute 'get_current_text'

In [ ]:
from transformers import TextStreamer
# Define messages
message1 = {"role": "system", "content": "Your name is Turk-GPT. You made by OmBayus."}
message = {"role": "user", "content": "who is OmBayus"}

# Apply chat template
system = tokenizer.apply_chat_template([message1], tokenize=False)
prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

# Combine system and prompt
inputs = tokenizer([system + prompt], return_tensors="pt").to("cuda")
#inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

# Enable fast inference
FastLanguageModel.for_inference(model)

# Define text streamer
text_streamer = TextStreamer(tokenizer)

# Generate response
_ = model.generate(**inputs, streamer=text_streamer,max_new_tokens=128)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<s><|im_start|> system
Your name is Turk-GPT. You made by OmBayus.<|im_end|> 
<|im_start|> user
who is OmBayus<|im_end|> 
<|im_start|> assistant
OmBayus is an AI developer or a user who has created or interacted with various AI models, including the one that generated the persona "Turk-GPT." The name "OmBayus" might be a pseudonym, username, or simply a combination of words to represent the individual's identity.<|im_end|>


In [ ]:
%%writefile app.py
import streamlit as st
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "OmBayus/Turk-GPT-v0.2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    # load_in_4bit = load_in_4bit,
    #token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Define the messages
message1 = {"role": "system", "content": "Your name is Turk-GPT. You made by OmBayus."}

def get_response(user_input):
    # Apply chat template
    system = tokenizer.apply_chat_template([message1], tokenize=False)
    prompt = tokenizer.apply_chat_template([{"role": "user", "content": user_input}], tokenize=False, add_generation_prompt=True)

    # Combine system and prompt
    inputs = tokenizer([system + prompt], return_tensors="pt").to("cuda")

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=128)

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract and return only the assistant's final response
    assistant_response = response.split("assistant")[-1].strip()
    return assistant_response

# Streamlit app layout
st.title("Turk-GPT Chatbot")
st.write("Ask me anything!")

user_input = st.text_input("Senin ismin nedir?")

if st.button("Send"):
    if user_input:
        response = get_response(user_input)
        st.write(f"Turk-GPT: {response}")
    else:
        st.write("Please enter a message.")



Overwriting app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.274s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!curl https://loca.lt/mytunnelpassword # tunnel password

35.197.145.231

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.14s
your url is: https://dark-chefs-cry.loca.lt
^C


### Saving to float16 for VLLM

In [ ]:
# Merge to 16bit
model.push_to_hub_merged("OmBayus/Turk-GPT-v0.2", tokenizer, save_method = "merged_16bit", token = "")

Unsloth: You are pushing to hub, but you passed your HF username = OmBayus.
We shall truncate OmBayus/Turk-GPT-v0.2 to Turk-GPT-v0.2
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 14.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 50.43 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 53.26it/s]


Unsloth: Saving tokenizer...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

Done.
Saved merged model to https://huggingface.co/OmBayus/Turk-GPT-v0.2
